In [ ]:
import torch
import torchaudio
from torch.utils.data import Dataset
from cnn import CNNNetwork
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle
class_mapping = [
    "air_conditioner",
    "car_horn",
    "children_playing",
    "dog_bark",
    "drilling",
    "engine_idling",
    "gun_shot",
    "jackhammer",
    "siren",
    "street_music"
]

class UrbanSoundDataset_load_to_predict(Dataset):

    def __init__(self,
                 audio_sample_path,
                 transformation,
                 target_sample_rate,
                 num_samples):
        self.audio_sample_path = audio_sample_path
        self.transformation = transformation
        self.target_sample_rate = target_sample_rate
        self.num_samples = num_samples


    def __getitem__(self, audio_sample_path):
        signal, sr = torchaudio.load(audio_sample_path)
        signal = self._resample_if_necessary(signal, sr)
        signal = self._mix_down_if_necessary(signal)
        signal = self._cut_if_necessary(signal)
        signal = self._right_pad_if_necessary(signal)
        signal = self.transformation(signal)
        return signal

    def _cut_if_necessary(self, signal):
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]
        return signal

    def _right_pad_if_necessary(self, signal):
        length_signal = signal.shape[1]
        if length_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

    def _resample_if_necessary(self, signal, sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        return signal

    def _mix_down_if_necessary(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal

#     def _get_audio_sample_path(self, index):
#         fold = f"fold{self.annotations.iloc[index, 5]}"
#         path = os.path.join(self.audio_dir, fold, self.annotations.iloc[
#             index, 0])
#         return path

    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 6]
    
def predict(model, input, class_mapping):
    model.eval()
    with torch.inference_mode():
        predictions = model(input)
        # Tensor (1, 10) -> [ [0.1, 0.01, ..., 0.6] ]
        predicted_index = predictions[0].argmax(0)
        predicted = class_mapping[predicted_index]
        expected = class_mapping[target]
    return predicted, expected

cnn = CNNNetwork()
state_dict = torch.load("cnnnet.pth")
cnn.load_state_dict(state_dict)
    
SAMPLE_RATE=16000
NUM_SAMPLES = 22050

mel_spec = torchaudio.transforms.MelSpectrogram(sample_rate = SAMPLE_RATE,
                                              n_fft = 1024,
                                              hop_length = 512,
                                              n_mels = 64)



flask_app = Flask(__name__)

@flask_app.route("/")
def Home():
    return "Hii"

@flask_app.route("/predict", methods = ["POST"])
def predict():
    audio_sample_path=request.files['audio']
    signal = UrbanSoundDataset_load_to_predict(r"E:\DL_audio\with_pytorch\UrbanSound8K\audio\fold1\7061-6-0-0.wav",mel_spec,SAMPLE_RATE,NUM_SAMPLES)
    signal1 = signal.__getitem__(r"E:\DL_audio\with_pytorch\UrbanSound8K\audio\fold1\7061-6-0-0.wav")
    prediction = cnn(signal1)
    predicted_index = predictions[0].argmax(0)
    prediction = class_mapping[predicted_index]
    return render_template("index.html", prediction_text = "The flower species is {}".format(prediction))


flask_app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2023-08-03 23:25:20,773] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "C:\Users\Jay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Jay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Jay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\flask\app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Jay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\fla